In [13]:
import numpy as np
import pandas as pd
import random

In [14]:
file_path = './data/'

In [15]:
currency_pair = 'Gbp_Chf'

In [16]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_H4_2022-2023.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [17]:
def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()


def atr_bands(high, low, close, lookback=14, atr_multiplier=3):
    scaled_atr_vals = atr(high, low, close, lookback) * atr_multiplier
    lower_band = close - scaled_atr_vals
    upper_band = close + scaled_atr_vals

    return lower_band, upper_band


df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['lower_atr_band'], df['upper_atr_band'] = atr_bands(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [18]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,atr,lower_atr_band,upper_atr_band
0,2022-06-17 09:00:00,1.18907,1.19127,1.18093,1.18156,1.18929,1.19150,1.18108,1.18173,1.18918,1.19138,1.18100,1.18164,35052,0.008466,1.156241,1.207039
1,2022-06-17 13:00:00,1.18155,1.18653,1.17937,1.18404,1.18171,1.18668,1.17958,1.18419,1.18163,1.18660,1.17948,1.18412,43446,0.008518,1.158566,1.209674
2,2022-06-17 17:00:00,1.18400,1.18700,1.18354,1.18406,1.18415,1.18721,1.18373,1.18556,1.18408,1.18710,1.18364,1.18481,20519,0.008238,1.160096,1.209524
3,2022-06-19 21:00:00,1.18431,1.18606,1.18283,1.18492,1.18767,1.18914,1.18446,1.18513,1.18599,1.18702,1.18416,1.18502,14015,0.008124,1.160649,1.209391
4,2022-06-20 01:00:00,1.18493,1.18519,1.18059,1.18301,1.18513,1.18543,1.18079,1.18322,1.18503,1.18530,1.18070,1.18312,22691,0.007990,1.159150,1.207090


In [19]:
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,atr,lower_atr_band,upper_atr_band
1544,2023-06-14 13:00:00,1.13929,1.14029,1.13807,1.13836,1.13946,1.14047,1.13827,1.13856,1.13938,1.14038,1.13818,1.13846,14957,0.002404,1.131247,1.145673
1545,2023-06-14 17:00:00,1.13835,1.14148,1.13784,1.14083,1.13852,1.14165,1.13818,1.14109,1.13844,1.14156,1.13810,1.14096,25268,0.002456,1.133591,1.148329
1546,2023-06-14 21:00:00,1.14085,1.14085,1.13973,1.14023,1.14129,1.14189,1.14012,1.14042,1.14107,1.14114,1.14002,1.14032,2576,0.002362,1.133234,1.147406
1547,2023-06-15 01:00:00,1.14025,1.14242,1.14018,1.14225,1.14042,1.14262,1.14040,1.14248,1.14034,1.14252,1.14030,1.14236,6927,0.002407,1.135139,1.149581
1548,2023-06-15 05:00:00,1.14226,1.14524,1.14215,1.14350,1.14248,1.14545,1.14238,1.14367,1.14237,1.14534,1.14226,1.14358,11013,0.002541,1.135958,1.151202


In [20]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [21]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100

In [22]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units


def run_simulation(spread_cutoff, n_bars, pips_to_risk, invert, start_hour, end_hour, sl_multiplier, atr_multiplier):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    win_amounts, loss_amounts, pips_risked = [], [], []
    day_fees = 0
    trade = None
    pips_to_risk = pips_to_risk / divider if type(pips_to_risk) != str else pips_to_risk
    lookback = 12 if n_bars == 'any' else n_bars
    lookforward = 0

    for i in range(12, len(df)):
        curr_bid_open, curr_bid_low, curr_ask_open, curr_ask_high, curr_mid_open, curr_ask_close, curr_bid_close, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Mid_Open', 'Ask_Close', 'Bid_Close', 'Date']]
        within_time_slot = start_hour <= curr_date.hour <= end_hour

        if trade is None and within_time_slot:
            spread = abs(curr_ask_open - curr_bid_open)
            atr, atr_lb, atr_ub = df.loc[df.index[i - 1], ['atr', 'lower_atr_band', 'upper_atr_band']]
            pip_movement = atr * atr_multiplier
            mid_highs = list(df.loc[df.index[i - 12:i], 'Mid_High'])
            mid_lows = list(df.loc[df.index[i - 12:i], 'Mid_Low'])

            if n_bars == 'any':
                mid_opens = list(df.loc[df.index[i - lookback:i], 'Mid_Open'])

                mid_closes = list(df.loc[df.index[i - lookback:i], 'Mid_Close'])

                def _check_bars(pips_moved, buy):
                    for j in range(len(mid_opens) - 1, -1, -1):
                        if (buy and mid_opens[j] < mid_closes[j]) or (not buy and mid_opens[j] > mid_closes[j]):
                            pips_moved += abs(mid_opens[j] - mid_closes[j])

                            if pips_moved >= pip_movement:
                                return True
                            
                        else:
                            return False
                            
                    return False

                buy_signal = _check_bars(0, buy=True)
                sell_signal = _check_bars(0, buy=False)

            else:
                mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
                mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

                buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
                sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

            if invert:
                buy_signal, sell_signal = sell_signal, buy_signal

            highest_high, lowest_low = max(mid_highs), min(mid_lows)

            if buy_signal:
                open_price = float(curr_ask_open)

                if pips_to_risk == 'bars':
                    curr_pips_to_risk = open_price - lowest_low

                elif pips_to_risk == 'atr':
                    curr_pips_to_risk = atr

                elif pips_to_risk == 'atr_bands':
                    curr_pips_to_risk = open_price - atr_lb

                else:
                    curr_pips_to_risk = pips_to_risk

                curr_pips_to_risk *= sl_multiplier
                stop_loss = open_price - curr_pips_to_risk
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price and spread <= curr_pips_to_risk * spread_cutoff:
                    n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                    trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 
                                'end_date': None, 'prev_profit_ratio': None}

                    n_buys += 1

                    pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(curr_bid_open)
                
                if pips_to_risk == 'bars':
                    curr_pips_to_risk = highest_high - open_price

                elif pips_to_risk == 'atr':
                    curr_pips_to_risk = atr

                elif pips_to_risk == 'atr_bands':
                    curr_pips_to_risk = atr_ub - open_price

                else:
                    curr_pips_to_risk = pips_to_risk

                curr_pips_to_risk *= sl_multiplier
                stop_loss = open_price + curr_pips_to_risk
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price and spread <= curr_pips_to_risk * spread_cutoff:
                    n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                    trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                            'pips_risked': round(curr_pips_to_risk, rounding),
                            'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 
                            'end_date': None, 'prev_profit_ratio': None}

                    n_sells += 1

                    pips_risked.append(curr_pips_to_risk)


        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_close > trade['open_price']:
            curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_close < trade['open_price']:
            curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts,

In [23]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
spread_cutoffs = [0.1]
n_bars_vals = ['any']  # LEAVE THE 'any' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
# pips_to_risk_vals = ['bars', 'atr', 'atr_bands', 20, 30]  # LEAVE THE 'bars' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pips_to_risk_vals = ['atr', 30]
invert_vals = [True]
# Hour values are between 0 (12 AM/midnight) and 23 (11 PM).  The dates are in UTC, which is about 7 hours ahead of CA time.
# For example, if you wanted to only place trades between 6 AM and 12 PM CA time, the start hour should be 13 (1 PM UTC) and
# the end hour should be 19 (7 PM UTC).  If you want to trade all 24 hours of the day, you would set the start hour to 0 and
# the end hour to 23.
start_hours = [0]
end_hours = [23]
# sl_multipliers = [1.0, 1.5, 2.0, 3.0]
sl_multipliers = [1.0]
atr_multipliers = [1.0, 1.5, 2.0]

all_combos = []

for spread_cutoff in spread_cutoffs:
    for n_bars in n_bars_vals:
        for pips_to_risk in pips_to_risk_vals:
            for invert in invert_vals:
                for start_hour in start_hours:
                    for end_hour in end_hours:
                        for sl_multiplier in sl_multipliers:
                            for atr_multiplier in atr_multipliers:
                                all_combos.append((spread_cutoff, n_bars, pips_to_risk, invert, start_hour, end_hour, sl_multiplier, atr_multiplier))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_spread_cutoff = None
best_n_bars_val = None
best_pips_to_risk = None
best_invert_val = None
best_start_hour = None
best_end_hour = None
best_sl_multiplier = None
best_atr_multiplier = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for spread_cutoff, n_bars, pips_to_risk, invert, start_hour, end_hour, sl_multiplier, atr_multiplier in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts, = run_simulation(spread_cutoff, n_bars, pips_to_risk, invert, start_hour, end_hour, sl_multiplier, atr_multiplier)
    runs_finished += 1

    print(reward, day_fees, reward + day_fees)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    total_profit = reward + day_fees

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or total_profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(total_profit), 'spread_cutoff': spread_cutoff, 'n_bars': n_bars, 'pips_to_risk': pips_to_risk, 'invert': invert, 'start_hour': start_hour, 'end_hour': end_hour, 'sl_multiplier': sl_multiplier, 'atr_multiplier': atr_multiplier})


    if total_profit > best_reward:
        best_reward = total_profit
        best_spread_cutoff = spread_cutoff
        best_n_bars_val = n_bars
        best_pips_to_risk = pips_to_risk
        best_invert_val = invert
        best_start_hour = start_hour
        best_end_hour = end_hour
        best_sl_multiplier = sl_multiplier
        best_atr_multiplier = atr_multiplier

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 6

386.4323800000035 -145.07886 241.3535200000035
Num buys: 32
Num sells: 41
Num trades: 73
Num wins: 17
Num losses: 41
Win streak: 5
Loss streak: 10
Avg pips risked: 0.0043377690802348335
Avg win amount: 158.72875529411755
Min win amount: 55.44213999999899
Max win amount: 608.9406400000001
Avg loss amount: -41.28493678571419
Min loss amount: -58.88000000000204
Max loss amount: 0.0
Remaining runs: 5
Best reward so far: 241.3535200000035

-1337.1299999999972 -122.05185999999999 -1459.1818599999972
Num buys: 35
Num sells: 47
Num trades: 82
Num wins: 11
Num losses: 55
Win streak: 5
Loss streak: 33
Avg pips risked: 0.0029999999999999996
Avg win amount: 160.01345454545375
Min win amount: 55.358999999998
Max win amount: 399.6089999999982
Avg loss amount: -43.62363380281674
Min loss amount: -58.88100000000223
Max loss amount: 0.0
Remaining runs: 4
Best reward so far: 241.3535200000035

-1484.6429999999657 -208.56778 -1693.2107799999658
Num buys: 63
Num sells: 70
Num trades: 133
Num 

In [24]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best invert val: ' + str(best_invert_val))
print('Best start hour: ' + str(best_start_hour))
print('Best end hour: ' + str(best_end_hour))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 241.3535200000035
Best spread: 0.1
Best n bars val: any
Best pips to risk: atr
Best invert val: True
Best start hour: 0
Best end hour: 23
Best sl multiplier: 1.0
Best atr multiplier: 2.0
-----------------------
Top results:
{'reward': 241, 'spread_cutoff': 0.1, 'n_bars': 'any', 'pips_to_risk': 'atr', 'invert': True, 'start_hour': 0, 'end_hour': 23, 'sl_multiplier': 1.0, 'atr_multiplier': 2.0}
{'reward': -1459, 'spread_cutoff': 0.1, 'n_bars': 'any', 'pips_to_risk': 30, 'invert': True, 'start_hour': 0, 'end_hour': 23, 'sl_multiplier': 1.0, 'atr_multiplier': 2.0}
{'reward': -1693, 'spread_cutoff': 0.1, 'n_bars': 'any', 'pips_to_risk': 30, 'invert': True, 'start_hour': 0, 'end_hour': 23, 'sl_multiplier': 1.0, 'atr_multiplier': 1.5}
{'reward': -1702, 'spread_cutoff': 0.1, 'n_bars': 'any', 'pips_to_risk': 'atr', 'invert': True, 'start_hour': 0, 'end_hour': 23, 'sl_multiplier': 1.0, 'atr_multiplier': 1.0}
{'reward': -490, 'spread_cutoff': 0

In [ ]:
# 'atr', 1.0: 4